# Harnessing Object and Scene Semantics for Large-Scale Video Understanding

[paper link](http://cs.brown.edu/~ls/Publications/cvpr2016wu.pdf)

## 动机

识别vedio中的目标（object）和场景语义（scene semantics）。

## 模型架构

包含了三种信息流的语义融合网络（semantic fusion network），分为三层：1.基于帧的低层次的CNN特征；2.基于目标（object）的大型CNN，用来识别对象，可识别20K个类；3.场景特征的大型CNN，可以识别205个场景。

使用object/scene 关系对 作为语义的表示（semantic representation）。

![](https://raw.githubusercontent.com/applenob/paper_note/master/res/hoss-model.png)
![](https://raw.githubusercontent.com/applenob/paper_note/master/res/hoss-model-2.png)



## 数学

训练数据：$D$，每段视频的类标是$z_i∈ Z_{Tr}$，$D=\{(x_i,z_i)\}_{i=1,...,n_{Tr}}$，其中，$n_{Tr}$是训练视频的个数；视频：$V_i=\{f_{i,1},...,f_{i,n_i}\}$，其中$n_i$是$V_i$的帧数。

1.Object Stream：使用VGG-19 CNN模型（使用ImageNet预训练好了，可以分类20574个目标），最后一个全连接层的输出作为融合网络的输入之一：第i个视频的j帧$f_{i,j} \rightarrow x_{i,j}^O∈R^{20574}$。

2.Scene Stream：使用VGG-16 CNN模型（使用Places205数据集预训练好了，包含205个场景和2.5m张图片）。同样使用最后一个全连接层的输出作为融合网络的输入之一：$f_{i,j} \rightarrow x_{i,j}^S∈R^{205}$。

3.Generic feature Stream：提取更通用的视觉信息，比如颜色等。使用VGG-19 CNN模型（使用ImageNet预训练好的），但是使用第一个全连接层的特征作为融合网络的输入之一：$f_{i,j} \rightarrow x_{i,j}^F∈R^{4096}$。

4.融合网络（Fusion Network）：使用前面三个网络的的输出作为输入，视频$V_i$的输入表示为：$\overline x_i^O=\sum_{k=1}^{n_i}x_{i,k}^O$，$\overline x_i^O=\sum_{k=1}^{n_i}x_{i,k}^O$，$\overline x_i^O=\sum_{k=1}^{n_i}x_{i,k}^O$。即，视频的特征是每帧特征的平均值。

预测：$\hat z_i = argmax_{z∈Z_{Tr}}f_z(\overline x_i)$，其中$f_z(\overline x_i)$是第i个视频$V_i$属于类别z的得分。

## OSR（Object and Scene semantic Representation）

令$k∈\{O,S\}$，在训练过程中，$\overline x_i^k$可以通过反向传播来优化，目标函数是：$f_z(\overline x_i)- λ||\overline x_i^k||_2$。优化后的是$\hat x_z^k$。这样，OSR矩阵是：$Π^k=\big[(\hat x_z^k)^T\big]_z$。这条式子可能表述有些歧义，实际上表示的是：$Π$第z列，即类标z的语义向量。

## Zero-shot Learning

令$Π = [Π^O , Π^S] $，用$Z_{Te}$来表示测试样例的类别，假设$Z_{Tr} ∩ Z_{Te}=∅$。

测试类的雏形：$Π_{\tilde z}=\sum_{z=1}^{|Z_{Tr}|}sim(\tilde z, z) \cdot Π_z$，其中，$\tilde z∈Z_{Te}$，$Π_z$是$Π$中对应类标z的一列，而$sim(\tilde z, z)$是两个类标的**语义相似度**，可以手动定义，使用WordNet或者词向量（作者使用word2vec）。

Zero-shot 识别：$\hat z = argmax_{\tilde z ∈ Z_{Te}}cos(g(V),Π_{\tilde z})$。 其中，$g(V)$是测试样例的表示，使用简单的最近邻方法预测zero-shot类标。

如何定义$g(V)$：构建伪实例原型：$g(V)=\frac{1}{∆}\sum_{t=1}^Tp(z_t|x̄)·Π_{z_t}$，其中$z_t$代表所有训练类标中，给定视频V概率第t大的类标，也就是$p(z_t|x̄)$；$∆=\sum_{t=1}^Tp(z_t|x̄)$是归一化因子。